In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import pandas as pd
import time

path = "/Users/tusharbector/Downloads/chromedriver" 
WEBSITES = {
    'langley': 'https://www.zoocasa.com/langley-bc-real-estate/sold?page=',
    'surrey': 'https://www.zoocasa.com/surrey-bc-real-estate/sold?page=',
    'burnaby': 'https://www.zoocasa.com/burnaby-bc-real-estate/sold?page=',
    'richmond': 'https://www.zoocasa.com/richmond-bc-real-estate/sold?page=',
    'vancouver': 'https://www.zoocasa.com/vancouver-bc-real-estate/sold?page=',
    'abbotsford': 'https://www.zoocasa.com/abbotsford-bc-real-estate/sold?page=',
    'coquitlam': 'https://www.zoocasa.com/coquitlam-bc-real-estate/sold?page=',
    'mapleRidge': 'https://www.zoocasa.com/maple-ridge-bc-real-estate/sold?page=',
    'delta': 'https://www.zoocasa.com/delta-bc-real-estate/sold?page=',
    'pittMeadows': 'https://www.zoocasa.com/pitt-meadows-bc-real-estate/sold?page=',
    'newWestminster': 'https://www.zoocasa.com/new-westminster-bc-real-estate/sold?page=',
    'whiteRock': 'https://www.zoocasa.com/white-rock-bc-real-estate/sold?page='
}

email = "bectort14@gmail.com"
password = "Zoocasa@987"

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)
logged_in = False

XPATHS = {
    'login': './/a[contains(@data-testid,"loginHeader")]',
    'email': './/input[contains(@data-testid,"loginRegistrationEmailTextField")]',
    'continue': './/button[contains(@data-testid,"loginRegistrationContinueButton")]',
    'password': './/input[contains(@data-testid,"passwordLoginModal")]',
    'signin': './/button[contains(@data-testid,"signInButton")]',
    'view': './/button[contains(@data-testid,"overlaySignInBtn")]',
    'tnc': './/button[contains(@data-testid,"termsOfUseAcceptButton_31017")]',
    'close': '#listing > div.style_bar__mddLI > div > svg > g > g > path',
    'listingCard': './/div[contains(@class,"style_wrapper__Z8WDF")]',
    'address': './/h1[contains(@data-testid,"streetAddress")]',
    'beds': './/span[contains(@data-testid,"listingBedIcon")]',
    'bath': './/span[contains(@data-testid,"listingBathIcon")]',
    'area': './/span[contains(@data-testid,"listingDimensionsIcon")]',
    'parking': './/span[contains(@data-testid,"listingCarIcon")]',
    'maintenance': './/span[contains(@data-testid,"MaintenanceFeesKeyFacts")]',
    'type': './/span[contains(@data-testid,"TypeKeyFacts")]',
    'taxes': './/span[contains(@data-testid,"TaxesKeyFacts")]',
    'built': './/span[contains(@data-testid,"Approx.AgeKeyFacts")]',
    'mls': './/span[contains(@data-testid,"MLS®NumberKeyFacts")]',
    'levels': './/span[contains(@data-testid,"LevelsKeyFacts")]',
    'garage': './/span[contains(@data-testid,"GarageKeyFacts")]',
    'lotSize': './/span[contains(@data-testid,"LotSizeKeyFacts")]',
    'pricePerSqFt': './/span[contains(@data-testid,"Avg.PricePerSqftKeyFacts")]',
    'soldDate': './/div[contains(@data-testid,"priceHistoryRowSoldDate")]',
    'soldPrice': './/div[contains(@data-testid,"priceHistoryRowSoldPrice")]',
    'listingDate': './/div[contains(@data-testid,"priceHistoryRowDate")]',
    'listingPrice': './/div[contains(@data-testid,"priceHistoryRowListPrice")]'
}

def login():
    try:
        loginBtn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['login'])))
        loginBtn.click()
        
        emailBtn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['email'])))
        emailBtn.send_keys(email)
        
        continueBtn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['continue'])))
        continueBtn.click()
        
        passwordBtn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['password'])))
        passwordBtn.send_keys(password)
        
        signInBtn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['signin'])))
        signInBtn.click()
        
        view = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['view'])))
        view.click()
        
        tnc = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, XPATHS['tnc'])))
        tnc.click()
        
        close = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, XPATHS['close'])))
        close.click()
        logged_in = True
    except TimeoutException:
        print("Login process timed out.")

def appendData(list, text):
    try:
        list.append(driver.find_element(By.XPATH, XPATHS[text]).text)
    except NoSuchElementException:
        list.append(None)

def getData(site):
    driver.get(site)
    login()

    listingsData = pd.DataFrame(columns=[
        'address', 'beds', 'bath', 'area', 'maintenance', 'parking', 'taxes', 
        'built', 'mlsNumber', 'levels', 'garage', 'lotSize', 
        'pricePerSqFt', 'soldDate', 'soldPrice', 'listingDate', 'listingPrice'
    ])
        
    for page_num in range(4, 6):
        if(True):
            driver.get(site + str(page_num))
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1) 
                
            listingCards = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, XPATHS['listingCard'])))
                
            for card in listingCards:
                try:
                    card.click()
                    time.sleep(1)
                            
                    address_data = []
                    bed_data = []
                    bath_data = []
                    area_data = []
                    parking_data = []
                    taxes_data = []
                    built_data = []
                    mlsNumber_data = []
                    levels_data = []
                    garage_data = []
                    lotSize_data = []
                    pricePerSqFt_data = []
                    soldDate_data = []
                    soldPrice_data = []
                    listingDate_data = []
                    listingPrice_data = []
                    maintenance_data = []

                    appendData(address_data, 'address')
                    appendData(bed_data, 'beds')
                    appendData(bath_data, 'bath')
                    appendData(area_data, 'area')
                    appendData(parking_data, 'parking')
                    appendData(taxes_data, 'taxes')
                    appendData(built_data, 'built')
                    appendData(mlsNumber_data, 'mls')
                    appendData(levels_data, 'levels')
                    appendData(garage_data, 'garage')
                    appendData(lotSize_data, 'lotSize')
                    appendData(pricePerSqFt_data, 'pricePerSqFt')
                    appendData(soldDate_data, 'soldDate')
                    appendData(soldPrice_data, 'soldPrice')
                    appendData(listingDate_data, 'listingDate')
                    appendData(listingPrice_data, 'listingPrice')
                    appendData(maintenance_data, 'maintenance')

                    new_data = pd.DataFrame({
                        'address': address_data,
                        'beds': bed_data,
                        'bath': bath_data,
                        'area': area_data,
                        'maintenance': maintenance_data,
                        'parking': parking_data,
                        'taxes': taxes_data,
                        'built': built_data,
                        'mlsNumber': mlsNumber_data,
                        'levels': levels_data,
                        'garage': garage_data,
                        'lotSize': lotSize_data,
                        'pricePerSqFt': pricePerSqFt_data,
                        'soldDate': soldDate_data,
                        'soldPrice': soldPrice_data,
                        'listingDate': listingDate_data,
                        'listingPrice': listingPrice_data
                    })

                    listingsData = pd.concat([listingsData, new_data], ignore_index=True)

                    close = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, XPATHS['close'])))
                    close.click()
                    time.sleep(1)  # Give some time for the modal to close

                except (NoSuchElementException, TimeoutException, WebDriverException) as e:
                    print(f"An error occurred with a listing card: {e}")
                    driver.get(site + str(page_num))
                    time.sleep(1)
                    continue  # Skip to the next listing card

    driver.quit()
    return listingsData


In [2]:
data = getData(WEBSITES['langley'])

In [3]:
data

,address,beds,bath,area,maintenance,parking,taxes,built,mlsNumber,levels,garage,lotSize,pricePerSqFt,soldDate,soldPrice,listingDate,listingPrice
0,"154 - 8737 212 Street, Langley, BC",3,2,1597 sqft,None,2,"$3,738",1990,R2891299,Residential,Yes,,$525,12/06/2024,"$902,000",05/06/2024,"$839,000"
1,"16 - 4426 232 Street, Langley, BC",2,1,812 sqft,None,2,$368,1970,R2870040,Residential,No,,$344,12/06/2024,"$259,000",14/04/2024,"$279,900"
2,"44 - 20760 Duncan Way, Langley, BC",3,2,1378 sqft,None,2,"$2,128",2004,R2890430,Residential,Yes,,$558,12/06/2024,"$743,000",03/06/2024,"$769,800"
3,"191 - 8288 207 A Street, Langley, BC",2,2,1102 sqft,,2,"$3,182",2013,R2882136,Residential,Yes,None,$680,11/06/2024,"$720,000",12/05/2024,"$749,900"
4,"208 - 19945 Brydon Crescent, Langley, BC",2,2,871 sqft,,1,"$1,767",2023,R2886246,Residential,Yes,None,$688,11/06/2024,"$597,000",21/05/2024,"$599,900"
5,"103 - 20897 57 Avenue, Langley, BC",2,2,1228 sqft,,1,"$2,542",1994,R2884514,Residential,Yes,None,$479,11/06/2024,"$586,300",16/05/2024,"$588,800"
6,"211 - 5363 206 Th Street, Langley, BC",2,2,1176 sqft,,1,"$2,031",1989,R2878586,Residential,Yes,None,$365,11/06/2024,"$425,000",02/05/2024,"$429,900"
7,"217 - 20200 56 Avenue, Langley, BC",3,2,976 sqft,,1,,1997,R2881817,Residential,Yes,None,$599,11/06/2024,"$578,000",13/05/2024,"$585,000"
8,"48 - 8068 207 Street, Langley, BC",4,3,1770 sqft,None,2,"$4,004",2014,R2889565,Residential,Yes,,$505,09/06/2024,"$890,000",02/06/2024,"$895,000"
9,"301 - 20245 53 Avenue, Langley, BC",1,1,739 sqft,,1,"$2,075",1996,R2888808,Residential,Yes,None,$608,09/06/2024,"$443,000",03/06/2024,"$449,900"
